In [ ]:
import asyncio
import logging
import sys

from lsst.ts import salobj

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
async def run_lifecycle_command(csc, command_str, settings_to_apply=None):
    try:
        cmd = getattr(csc, f"cmd_{command_str}")
        if command_str == "start" and settings_to_apply is not None:
            cmd.set(settingsToApply=settings_to_apply)
        await cmd.start(timeout=15)
    except salobj.AckError as ae:
        print(f"Failed with ack={ae.ackcmd.ack}, ack.error={ae.ackcmd.error}, ackcmd.result={ae.ackcmd.result}")

In [ ]:
d = salobj.Domain()

CCArchiver = salobj.Remote(d, 'CCArchiver')
CCCamera = salobj.Remote(d, 'CCCamera') 
CCHeaderService = salobj.Remote(d, 'CCHeaderService')

await asyncio.gather(CCArchiver.start_task, CCHeaderService.start_task, CCCamera.start_task)

In [ ]:
#command = "start"
command = "enable"
#command = "disable"
#command = "standby"
#command = "exitControl"

In [ ]:
await run_lifecycle_command(CCArchiver, command, "normal")

In [ ]:
await run_lifecycle_command(CCHeaderService, command)

In [ ]:
await run_lifecycle_command(CCCamera, command, "Normal")